In [1]:
from collections import defaultdict
import glob
import os
import re
import xml.etree.ElementTree as et

XML_FOLDER = '/Users/pbatra/broad/local_data_sets/tensors/test/ecg-rest-and-bike/2019-05-23/raw'
SAMPLE_ID='1994810_6025_0_0.xml'
xml_path = os.path.join(XML_FOLDER, SAMPLE_ID)
print(xml_path)
ecg = glob.glob(xml_path)
root = et.parse(ecg[0]).getroot()

/Users/pbatra/broad/local_data_sets/tensors/test/ecg-rest-and-bike/2019-05-23/raw/1994810_6025_0_0.xml


In [38]:
#protocol
protocols = defaultdict(dict)
for protocol in root.findall("./Protocol/Phase"):
    phase_name = protocol.find("PhaseName").text
    phase_duration = 60 * int(protocol.find("PhaseDuration/Minute").text) + int(protocol.find("PhaseDuration/Second").text)
    print(phase_name, phase_duration)

Pretest 15
Exercise 360
Rest 60


In [32]:
#[(phase_name,stage,duration)]
trends = defaultdict(dict)
trend_entry_fields=['HeartRate', 'Load', 'Grade', 'StageName', 'PhaseName', 'Mets', 'VECount', 'PaceCount', 'Artifact']

for trend_entry in root.findall("./TrendData/TrendEntry"):
    idx = int(trend_entry.get("Idx"))
    for field in trend_entry_fields:
        trends[idx][field]=trend_entry.find(field).text
    current_trend_start_time = 60 * int(trend_entry.find("EntryTime/Minute").text) + int(trend_entry.find("EntryTime/Second").text)
    trends[idx]['start'] = current_trend_start_time
    if idx > 0:
        trends[idx-1]['end'] = current_trend_start_time
    #last trend has no end
    
#now, let's go through and read out the full disclosure data
print(trends)

defaultdict(<class 'dict'>, {0: {'HeartRate': '70', 'Load': '0', 'Grade': '0.0', 'StageName': 'Steady', 'PhaseName': 'Pretest', 'Mets': '1.0', 'VECount': '0', 'PaceCount': '0', 'Artifact': '0%', 'start': 0, 'end': 15}, 1: {'HeartRate': '72', 'Load': '0', 'Grade': '0.0', 'StageName': 'Steady', 'PhaseName': 'Pretest', 'Mets': '1.0', 'VECount': '0', 'PaceCount': '0', 'Artifact': '0%', 'start': 15, 'end': 16}, 2: {'HeartRate': '72', 'Load': '0', 'Grade': '0.0', 'StageName': 'Constant', 'PhaseName': 'Exercise', 'Mets': '1.0', 'VECount': '0', 'PaceCount': '0', 'Artifact': '0%', 'start': 16, 'end': 24}, 3: {'HeartRate': '74', 'Load': '40', 'Grade': '6.1', 'StageName': 'Constant', 'PhaseName': 'Exercise', 'Mets': '1.1', 'VECount': '0', 'PaceCount': '0', 'Artifact': '0%', 'start': 24, 'end': 29}, 4: {'HeartRate': '81', 'Load': '40', 'Grade': '6.3', 'StageName': 'Constant', 'PhaseName': 'Exercise', 'Mets': '1.2', 'VECount': '0', 'PaceCount': '0', 'Artifact': '0%', 'start': 29, 'end': 44}, 5: {'H

In [33]:
#TODO: start_time, sample_rate
counter = 0
full_ekgs = defaultdict(list)
full_leads = {}
fd_start_time = (60 * int(root.find("./FullDisclosure/StartTime/Minute").text) + 
                   float(root.find("./FullDisclosure/StartTime/Second").text))
fd_sample_rate = int(root.find("./FullDisclosure/SampleRate").text)
for lead_order in root.findall("./FullDisclosure/LeadOrder"):
    full_leads = {i: lead for i, lead in enumerate(lead_order.text.split(','))}
    
for full_d in root.findall("./FullDisclosure/FullDisclosureData"):
    for full_line in re.split('\n|\t', full_d.text):
        for sample in re.split(',', full_line):
            if sample == '':
                continue
            full_ekgs[full_leads[counter % 3]].append(float(sample))
            counter += 1


In [19]:
full_ekgs['I'][-10:]

[40.0, 41.0, 43.0, 42.0, 42.0, 38.0, 36.0, 29.0, 21.0, 21.0]

In [44]:
print((7*60+15)*500)
print(len(full_ekgs['I']))
print (.8*500)

217500
216500
400.0


In [36]:
#TODO TEST COMPLETED INFO
#NONE AT END
# 1 sec gap
fd_start_time = 0
for trend in trends.values():
    start_idx = int(fd_sample_rate * (trend['start'] - fd_start_time))
    end_idx = int(fd_sample_rate * (trend['end'] - fd_start_time)) if trend.get('end') else None
    trend['lead_I'] = full_ekgs['I'][start_idx:end_idx]
    trend['lead_2'] = full_ekgs['2'][start_idx:end_idx]
    trend['lead_3'] = full_ekgs['3'][start_idx:end_idx]

In [37]:
#checks
raw_len = len(full_ekgs['I'])
total_len=0
for trend in trends.values():
    assert len(trend['lead_I']) == len(trend['lead_2']) == len(trend['lead_3'])
    total_len += len(trend['lead_I'])

print((raw_len - total_len)/500)
assert raw_len == total_len


0.0


In [ ]:
    phase_name = phase.find('PhaseName').text
    for stage in phase.findall("./Stage"):
        idx = int(stage.get("idx"))
        stage_name = stage.find("StageName").text
        stage_duration =60 * int(stage.find("StageDuration/Minute").text) + int(stage.find("StageDuration/Second").text) 
        stage_end_time = stage_start_time + stage_duration
        print(phase_name, idx, stage_name, stage_duration, stage_start_time, stage_end_time)
       
        protocols[phase_name][idx] = {'stage_name': stage_name, 'stage_duration': stage_duration, 
                                                    'stage_start_time': stage_start_time, 'stage_end_time': stage_end_time}
        
        stage_start_time = stage_end_time


        

In [ ]:
for key, item in protocols.items():
    print(key,item)
    print('\n')